In [ ]:
import gradio as gr
import os
from dotenv import load_dotenv
import openai
import json

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
MODEL_GPT = 'gpt-4o-mini'

In [ ]:
system_prompt = """
    You are an AI that generates complete HTML webpages.

    Always return the response in JSON format, with the following structure:

    {
        "description": "Brief description of the page",
        "generated_html": "<html>...</html>"
    }

    Ensure the 'generated_html' contains a complete, valid HTML document with inline CSS and JavaScript.
  """

In [ ]:
def chat_reply(history):
    messages = [{"role": "system", "content": system_prompt}] + history
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        response_format={"type": "json_object"}
    )
    parsed_json = json.loads(response.choices[0].message.content)
    formated_reply = f'**{parsed_json["description"]}**\n\n```html\n{parsed_json["generated_html"]}\n```'
    history += [{"role":"assistant", "content":formated_reply}]
    return history, formated_reply

In [ ]:

with gr.Blocks() as ui:
    with gr.Row():
        chat = gr.Chatbot(height=500, type="messages")
        code = gr.Code(label="Code", language="html")
        view = gr.HTML(label="View")
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history.append({"role": "user", "content": message})
        return "", history

    def update_view(new_code):
        return new_code

    entry.submit(do_entry, inputs=[entry, chat], outputs=[entry, chat]).then(
        chat_reply, inputs=chat, outputs=[chat, code]
    )
    code.change(update_view, inputs=code, outputs=view)
    clear.click(lambda: None, inputs=None, outputs=chat, queue=False)

ui.launch(inbrowser=True)